In [ ]:
import numpy as np
import xarray as xr
import pickle
import xesmf as xe

In [ ]:
# Load quarterly degree data of gph and temperature
#infile = open("./500GPH/FULL500_ANOM_capgph1959.p", 'rb') 
infile = open("1959_NAOgph500_div10.p", 'rb') 
gph = pickle.load(infile)
infile.close()

In [ ]:
##indices
infile = open("era5_latindex.p", 'rb') 
lats = pickle.load(infile)
infile.close()

infile = open("era5_lonindex.p", 'rb') 
lons = pickle.load(infile)
infile.close()

infile = open("era5_timeindex.p", 'rb') 
time = pickle.load(infile)
infile.close()

In [ ]:
##save year index
year = [i for i in range(1959,2022)] #indicate years for data, needed for opening files

In [ ]:
##save lat lon indices
gph_lats = lats[40:241]
gph_lons = lons[320:681]
#temp_lats = lats[60:121]
#temp_lons = lons [760:901]

In [ ]:
##create array for gph ... 
g = xr.DataArray(data= gph, dims = ["year","day","lat","lon"],coords = dict(year = year,
                                                                                day = time,
                                                                                lat = gph_lats,
                                                                                lon = gph_lons))
g

In [ ]:
##create array for temp ... 
t = xr.DataArray(data= temp, dims = ["year","day","lat","lon"],coords = dict(year = year,
                                                                                day = time,
                                                                                lat = temp_lats,
                                                                                lon = temp_lons))
t

In [ ]:
# Create the target grid (single degree)
t_out = xr.Dataset({'year':(['year'], year[:]),
                    'day':(['day'], time[:]),
                    'lat': (['lat'], np.arange(75, 59, -1)),
                    'lon': (['lon'], np.arange(10, 46, 1))})

In [ ]:
t_out

In [ ]:
# Create the regridder
regridder = xe.Regridder(t, t_out, 'bilinear', periodic=True)

In [ ]:
# Regrid the data
t_single_degree = regridder(t)
t_single_degree

In [ ]:
g_out = xr.Dataset({'year':(['year'], year[:]),
                    'day':(['day'], time[:]),
                    'lat': (['lat'], np.arange(80, 30, -2.5)),
                    'lon': (['lon'], np.arange(-100, -10, 2.5))})

In [ ]:
# Create the regridder
regridder = xe.Regridder(g, g_out, 'bilinear', periodic=True)

In [ ]:
# Regrid the data
g_single_degree = regridder(g)
g_single_degree

In [ ]:
gph1 = g_single_degree.values
gph1.shape

In [ ]:
temp1 = t_single_degree.values
temp1.shape

In [ ]:
##remove leap days now
gph1 = np.delete(gph1,[120],1)
#temp1 = np.delete(temp1,[120],1)

In [ ]:
temp1.shape

In [ ]:
pickle.dump(gph1, open("gph500_NAO25deg.p", 'wb'))
#pickle.dump(temp1, open("eurtemp1000_singledeg.p", 'wb'))